In [2]:
%load_ext autotime

%load_ext autoreload
%autoreload 2

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 9.67 ms


In [3]:
import numpy as np
import scipy.sparse as sp
from datafile_methods.data_io import save_csv
DATA_PATH = '../data/'
PREDICTION_PATH = '../data/predictions/'

time: 111 ms


In [ ]:
from datafile_methods.data_io import load_datasets
from datafile_methods.data_processing import load_data

# Load datasets
folds, ratings, sample_submission = load_datasets()

k_fold = len(folds)

models = ['baseline',
          'knn_baseline_i',
          'knn_baseline_u',
          'mf_svd_sci',
          'nmf',
          'slope_one',
          'sur_svd',
          'mf_als_recommend',
          'mf_als']

# Load predictions for each fold and model
predictions = [[load_data('{p}model_{m}_te_{i}.csv'.format(
    p=PREDICTION_PATH, m=model, i=i)) for i in range(k_fold)] for model in models]

predictions_dict = dict(zip(models, predictions))

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
from prediction_methods.create_ensemble import evaluate_meta_features_ensemble
predictors_high_df, observations_high_df, predictors_low_df, observations_low_df = evaluate_meta_features_ensemble(ratings, folds, predictions_dict)

High support

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
poly = PolynomialFeatures(2)

In [ ]:
regularization_errors_h_tr = {}
regularization_errors_h_te = {}

for alpha in np.linspace(0,.001,9):
    rmse_errors_tr = np.zeros(k_fold)
    rmse_errors_te = np.zeros(k_fold)
    for i in range(k_fold):
        # Get training data
        preds_train = predictors_high_df[i][::2]
        train = poly.fit_transform(preds_train)
        vals_train = observations_high_df[i][::2]
        # Create and fit model
        clf = Ridge(alpha=alpha, normalize=True)
        clf.fit(train, vals_train)
        # Obtain predictions for training set
        pred_train = clf.predict(train)
        pred_train = np.clip(pred_train, 1.0, 5.0)
        # Compute train error
        rmse_train = sqrt(mean_squared_error(vals_train, pred_train))

        # Use remaining data for testing
        preds_test = predictors_high_df[i][1::2]
        test = poly.fit_transform(preds_test)
        vals_test = observations_high_df[i][1::2]
        # Obtain predictions for test setpred_high_sub
        pred_test = clf.predict(test)
        # Compute test error
        rmse_test = sqrt(mean_squared_error(vals_test, pred_test))
        
        # Save errors in arrays
        rmse_errors_tr[i] = rmse_train
        rmse_errors_te[i] = rmse_test
    # Save errors in dictionaries
    regularization_errors_h_tr[alpha] = rmse_errors_tr
    regularization_errors_h_te[alpha] = rmse_errors_te

plt.figure(figsize=(12,5))
pd.DataFrame(regularization_errors_h_tr).boxplot()
pd.DataFrame(regularization_errors_h_te).boxplot()
plt.show()

plt.figure(figsize=(12,5))
pd.DataFrame(regularization_errors_h_tr).boxplot()
plt.show()

plt.figure(figsize=(12,5))
pd.DataFrame(regularization_errors_h_te).boxplot()
plt.show()

# Best: alpha = .000125

Low support

In [ ]:
regularization_errors_l_tr = {}
regularization_errors_l_te = {}

for alpha in np.linspace(0,.001,9):
    rmse_errors_tr = np.zeros(k_fold)
    rmse_errors_te = np.zeros(k_fold)
    for i in range(k_fold):
        # Get training data
        preds_train = predictors_low_df[i][::2]
        train = poly.fit_transform(preds_train)
        vals_train = observations_low_df[i][::2]
        # Create and fit model
        clf = Ridge(alpha=alpha, normalize=True)
        clf.fit(train, vals_train)
        # Obtain predictions for training set
        pred_train = clf.predict(train)
        pred_train = np.clip(pred_train, 1.0, 5.0)
        # Compute train error
        rmse_train = sqrt(mean_squared_error(vals_train, pred_train))

        # Use remaining data for testing
        preds_test = predictors_low_df[i][1::2]
        test = poly.fit_transform(preds_test)
        vals_test = observations_low_df[i][1::2]
        # Obtain predictions for test set
        pred_test = clf.predict(test)
        # Compute test error
        rmse_test = sqrt(mean_squared_error(vals_test, pred_test))
        
        # Save errors in arrays
        rmse_errors_tr[i] = rmse_train
        rmse_errors_te[i] = rmse_test
    # Save errors in dictionaries
    regularization_errors_l_tr[alpha] = rmse_errors_tr
    regularization_errors_l_te[alpha] = rmse_errors_te

plt.figure(figsize=(12,5))
pd.DataFrame(regularization_errors_l_tr).boxplot()
pd.DataFrame(regularization_errors_l_te).boxplot()
plt.show()

plt.figure(figsize=(12,5))
pd.DataFrame(regularization_errors_l_tr).boxplot()
plt.show()

plt.figure(figsize=(12,35))
pd.DataFrame(regularization_errors_l_te).boxplot()
plt.show()

# Best: alpha = .000375

#### Generate submission

In [ ]:
# Load predictions of submission entries with each model
predictions_sub = [load_data('{p}model_{m}_sub.csv'.format(p=PREDICTION_PATH, m=model))
    for model in models]
predictions_sub_dict = dict(zip(models, predictions_sub))